# DX 704 Week 4 Project

This week's project will test the learning speed of linear contextual bandits compared to unoptimized approaches.
You will start with building a preference data set for evaluation, and then implement different variations of LinUCB and visualize how fast they learn the preferences.


The full project description, a template notebook and supporting code are available on GitHub: [Project 4 Materials](https://github.com/bu-cds-dx704/dx704-project-04).


## Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Part 1: Collect Rating Data

The file "recipes.tsv" in this repository has information about 100 recipes.
Make a new file "ratings.tsv" with two columns, recipe_slug (from recipes.tsv) and rating.
Populate the rating column with values between 0 and 1 where 0 is the worst and 1 is the best.
You can assign these ratings however you want within that range, but try to make it reflect a consistent set of preferences.
These could be your preferences, or a persona of your choosing (e.g. chocolate lover, bacon-obsessed, or sweet tooth).
Make sure that there are at least 10 ratings of zero and at least 10 ratings of one.


Hint: You may find it more convenient to assign raw ratings from 1 to 5 and then remap them as follows.

`ratings["rating"] = (ratings["rating_raw"] - 1) * 0.25`

Submit "ratings.tsv" in Gradescope.

## Part 2: Construct Model Input

Use your file "ratings.tsv" combined with "recipe-tags.tsv" to create a new file "features.tsv" with a column recipe_slug, a column bias which is hard-coded to one, and a column for each tag that appears in "recipe-tags.tsv".
The tag column in this file should be a 0-1 encoding of the recipe tags for each recipe.
[Pandas reshaping function methods](https://pandas.pydata.org/docs/user_guide/reshaping.html) may be helpful.

The bias column will make later LinUCB calculations easier since it will just be another dimension.

Hint: For later modeling steps, it will be important to have the feature data (inputs) and the rating data (target outputs) in the same order.
It is highly recommended to make sure that "features.tsv" and "ratings.tsv" have the recipe slugs in the same order.

In [1]:
# YOUR CHANGES HERE

import pandas as pd

# Load the data files
ratings_df = pd.read_csv('ratings.tsv', sep='\t')
recipe_tags_df = pd.read_csv('recipe-tags.tsv', sep='\t')

# Print info about the files for debugging
print("Ratings file shape:", ratings_df.shape)
print("Recipe-tags file shape:", recipe_tags_df.shape)
print("\nFirst few rows of ratings:")
print(ratings_df.head())
print("\nFirst few rows of recipe-tags:")
print(recipe_tags_df.head())

# Get unique tags to create columns
unique_tags = sorted(recipe_tags_df['recipe_tag'].unique())
print(f"\nFound {len(unique_tags)} unique tags")
print("First 20 tags:", unique_tags[:20])

# Create features dataframe starting with recipe_slug from ratings
# ensure same order for ratings
features_df = ratings_df[['recipe_slug']].copy()

# Add bias column (hard-coded to 1)
features_df['bias'] = 1

# Create a mapping of recipe_slug to set of tags
recipe_to_tags = {}
for _, row in recipe_tags_df.iterrows():
    recipe_slug = row['recipe_slug']
    tag = row['recipe_tag']
    if recipe_slug not in recipe_to_tags:
        recipe_to_tags[recipe_slug] = set()
    recipe_to_tags[recipe_slug].add(tag)

print(f"\nProcessed tags for {len(recipe_to_tags)} recipes")

# Create binary encoding for each tag
for tag in unique_tags:
    # Create binary column: 1 if recipe has tag, 0 otherwise
    features_df[tag] = features_df['recipe_slug'].apply(
        lambda recipe: 1 if recipe in recipe_to_tags and tag in recipe_to_tags[recipe] else 0
    )

# Verify the order matches ratings.tsv
print(f"\nFeatures file will have shape: {features_df.shape}")
print("Columns:", list(features_df.columns[:10]), "... (showing first 10)")

# Check that recipe order matches between files
recipes_match = (features_df['recipe_slug'] == ratings_df['recipe_slug']).all()
print(f"\nRecipe order matches between features and ratings: {recipes_match}")

# Save to TSV file
features_df.to_csv('features.tsv', sep='\t', index=False)
print("\nSaved features.tsv successfully!")

Ratings file shape: (100, 2)
Recipe-tags file shape: (752, 2)

First few rows of ratings:
        recipe_slug  rating
0           falafel    0.25
1        spamburger    0.00
2  bacon-fried-rice    1.00
3   chicken-fingers    0.25
4       apple-crisp    0.75

First few rows of recipe-tags:
   recipe_slug recipe_tag
0  spam-musubi   hawaiian
1  spam-musubi       nori
2  spam-musubi    onthego
3  spam-musubi       rice
4  spam-musubi      snack

Found 296 unique tags
First 20 tags: ['alfredo', 'almond', 'american', 'appetizer', 'appetizers', 'apple', 'asiancuisine', 'asparagus', 'avocado', 'bacon', 'baked', 'bakeddishes', 'bakery', 'baking', 'beans', 'beef', 'bellpeppers', 'berries', 'blackbeans', 'blackvinegar']

Processed tags for 100 recipes

Features file will have shape: (100, 298)
Columns: ['recipe_slug', 'bias', 'alfredo', 'almond', 'american', 'appetizer', 'appetizers', 'apple', 'asiancuisine', 'asparagus'] ... (showing first 10)

Recipe order matches between features and ratings:

/tmp/ipykernel_2011/4016882601.py:43: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  features_df[tag] = features_df['recipe_slug'].apply(
/tmp/ipykernel_2011/4016882601.py:43: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  features_df[tag] = features_df['recipe_slug'].apply(
/tmp/ipykernel_2011/4016882601.py:43: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To ge

Submit "features.tsv" in Gradescope.

## Part 3: Linear Preference Model

Use your feature and rating files to build a ridge regression model with ridge regression's regularization parameter $\alpha$ set to 1.


Hint: If you are using scikit-learn modeling classes, you should use `fit_intercept=False` since that intercept value will be redundant with the bias coefficient.

Hint: The estimate component of the bounds should match the previous estimate, so you should be able to just focus on the variance component of the bounds now.

In [2]:
# YOUR CHANGES HERE

import pandas as pd
from sklearn.linear_model import Ridge
import numpy as np

# Load the feature and rating files
features_df = pd.read_csv('features.tsv', sep='\t')
ratings_df = pd.read_csv('ratings.tsv', sep='\t')

print("Features file shape:", features_df.shape)
print("Ratings file shape:", ratings_df.shape)

# Verify that recipe order matches between files
recipes_match = (features_df['recipe_slug'] == ratings_df['recipe_slug']).all()
print(f"Recipe order matches: {recipes_match}")

if not recipes_match:
    print("ERROR: Recipe orders don't match between files!")
    exit()

# Prepare the data for modeling
# X: feature matrix (excluding recipe_slug column)
# y: target ratings

# Get feature columns (everything except recipe_slug)
feature_columns = [col for col in features_df.columns if col != 'recipe_slug']
X = features_df[feature_columns].values
y = ratings_df['rating'].values

print(f"Feature matrix shape: {X.shape}")
print(f"Target vector shape: {y.shape}")
print(f"Number of features: {len(feature_columns)}")

# Display feature column names (first few and last few)
print(f"Feature columns: {feature_columns[:5]} ... {feature_columns[-5:]}")

# Build ridge regression model with α = 1
# fit_intercept=False already have a bias column
ridge_model = Ridge(alpha=1.0, fit_intercept=False)

# Fit the model
print("\nFitting ridge regression model...")
ridge_model.fit(X, y)

print("Model fitted successfully!")

# Get coefficients
coefficients = ridge_model.coef_

print(f"Number of coefficients: {len(coefficients)}")
print(f"Coefficient range: [{coefficients.min():.4f}, {coefficients.max():.4f}]")

# Create dataframe with feature names and coefficients
coef_df = pd.DataFrame({
    'feature_name': feature_columns,
    'coefficient': coefficients
})

coef_df_with_bias = coef_df.copy()  

# Rename columns to match required output format
coef_df_with_bias.columns = ['recipe_tag', 'coefficient']

print(f"\nCoefficients: {len(coef_df_with_bias)}")
print("Top 10 positive coefficients:")
print(coef_df_with_bias.sort_values('coefficient', ascending=False).head(10))

print("\nTop 10 negative coefficients:")
print(coef_df_with_bias.sort_values('coefficient', ascending=True).head(10))


Features file shape: (100, 298)
Ratings file shape: (100, 2)
Recipe order matches: True
Feature matrix shape: (100, 297)
Target vector shape: (100,)
Number of features: 297
Feature columns: ['bias', 'alfredo', 'almond', 'american', 'appetizer'] ... ['vegetarian', 'warm', 'whippedcream', 'winter', 'yeastdough']

Fitting ridge regression model...
Model fitted successfully!
Number of coefficients: 297
Coefficient range: [-0.1346, 0.3552]

Coefficients: 297
Top 10 positive coefficients:
    recipe_tag  coefficient
0         bias     0.355173
10       bacon     0.335862
88     dessert     0.232927
51   chocolate     0.177956
195      pasta     0.122409
14      baking     0.112239
178     nachos     0.095671
27   breakfast     0.090172
11       baked     0.085564
245    souffle     0.083602

Top 10 negative coefficients:
            recipe_tag  coefficient
292         vegetarian    -0.134633
204  pickledvegetables    -0.132536
139            healthy    -0.100077
130            gruyere    -0.

Save the coefficients of this model in a file "model.tsv" with columns "recipe_tag" and "coefficient".
Do not include the bias.

In [3]:
# YOUR CHANGES HERE

coef_df_with_bias.to_csv('model.tsv', sep='\t', index=False)
print(f"\nSaved model.tsv with {len(coef_df_with_bias)} coefficients")


Saved model.tsv with 297 coefficients


Submit "model.tsv" in Gradescope.

## Part 4: Recipe Estimates

Use the recipe model to estimate the score of every recipe.
Save these estimates to a file "estimates.tsv" with columns recipe_slug and score_estimate.

In [16]:
# YOUR CHANGES HERE

import pandas as pd
from sklearn.linear_model import Ridge
import numpy as np

# Load the necessary files
features_df = pd.read_csv('features.tsv', sep='\t')
ratings_df = pd.read_csv('ratings.tsv', sep='\t')
model_df = pd.read_csv('model.tsv', sep='\t')

print("Features file shape:", features_df.shape)
print("Ratings file shape:", ratings_df.shape)
print("Model file shape:", model_df.shape)

# Recreate the ridge regression model

# Prepare the data for modeling (same as Part 3)
feature_columns = [col for col in features_df.columns if col != 'recipe_slug']
X = features_df[feature_columns].values
y = ratings_df['rating'].values

# Refit the ridge model (same as Part 3)
ridge_model = Ridge(alpha=1.0, fit_intercept=False)
ridge_model.fit(X, y)

print("Ridge model refitted successfully")

# Use the model to predict scores for all recipes
all_predictions = ridge_model.predict(X)

print(f"Generated {len(all_predictions)} predictions")
print(f"Prediction range: [{all_predictions.min():.4f}, {all_predictions.max():.4f}]")

# Create estimates dataframe
estimates_df = pd.DataFrame({
    'recipe_slug': features_df['recipe_slug'],
    'score_estimate': all_predictions
})

print("First few estimates:")
print(estimates_df.head(10))

# Sort by score estimate to see highest and lowest predicted preferences
print("\nTop 10 highest estimated scores:")
top_estimates = estimates_df.sort_values('score_estimate', ascending=False).head(10)
print(top_estimates)

print("\nTop 10 lowest estimated scores:")
bottom_estimates = estimates_df.sort_values('score_estimate', ascending=True).head(10)
print(bottom_estimates)

# Verify predictions match actual ratings for training data
mse = np.mean((y - all_predictions) ** 2)
print(f"\nTraining MSE (should match Part 3): {mse:.6f}")

Features file shape: (100, 298)
Ratings file shape: (100, 2)
Model file shape: (296, 2)
Ridge model refitted successfully
Generated 100 predictions
Prediction range: [0.0025, 1.0368]
First few estimates:
                    recipe_slug  score_estimate
0                       falafel        0.229707
1                    spamburger        0.034134
2              bacon-fried-rice        0.960169
3               chicken-fingers        0.277268
4                   apple-crisp        0.782425
5         cranberry-apple-crisp        0.714080
6  bacon-chocolate-chip-cookies        1.036802
7                        sujebi        0.072551
8               pasta-primavera        0.285169
9                         ramen        0.033360

Top 10 highest estimated scores:
                     recipe_slug  score_estimate
6   bacon-chocolate-chip-cookies        1.036802
40            maple-bacon-donuts        1.016086
37  chocolate-peanut-butter-cake        1.011836
26  brioche-bread-with-chocolate      

In [17]:
# Save
estimates_df.to_csv('estimates.tsv', sep='\t', index=False)
print(f"\nSaved estimates.tsv with {len(estimates_df)} recipe estimates")


Saved estimates.tsv with 100 recipe estimates


Submit "estimates.tsv" in Gradescope.

## Part 5: LinUCB Bounds

Calculate the upper bounds of LinUCB using data corresponding to trying every recipe once and receiving the rating in "ratings.tsv" as the reward.
Keep the ridge regression regularization parameter at 1, and set LinUCB's $\alpha$ parameter to 2.
Save these upper bounds to a file "bounds.tsv" with columns recipe_slug and score_bound.

In [3]:
import pandas as pd
import numpy as np

Xdf = pd.read_csv("features.tsv", sep="\t")
rdf = pd.read_csv("ratings.tsv", sep="\t")

X = Xdf.set_index("recipe_slug")
y = rdf.set_index("recipe_slug").loc[X.index, "rating"].to_numpy()

Xmat = X.to_numpy(dtype=float)

alpha_ridge = 1.0
theta = np.linalg.solve(Xmat.T @ Xmat + alpha_ridge * np.eye(Xmat.shape[1]), Xmat.T @ y)

A = Xmat.T @ Xmat + alpha_ridge * np.eye(Xmat.shape[1])
A_inv = np.linalg.inv(A)
s2 = np.einsum("ij,jk,ik->i", Xmat, A_inv, Xmat)
s = np.sqrt(np.maximum(s2, 0.0))

alpha_linucb = 2.0

ucb = Xmat @ theta + alpha_linucb * s

bounds = pd.DataFrame({
    "recipe_slug": X.index,
    "score_bound": ucb
})
bounds.to_csv("bounds.tsv", sep="\t", index=False)

print(f"Saved bounds.tsv with shape={bounds.shape}")
print(bounds.head())

Saved bounds.tsv with shape=(100, 2)
        recipe_slug  score_bound
0           falafel     1.995962
1        spamburger     1.928470
2  bacon-fried-rice     2.826326
3   chicken-fingers     2.071384
4       apple-crisp     2.558767


Submit "bounds.tsv" in Gradescope.

## Part 6: Make Online Recommendations

Implement LinUCB to make 100 recommendations starting with no data and using the same parameters as in part 5.
One recommendation should be made at a time and you can break ties arbitrarily.
After each recommendation, use the rating from part 1 as the reward to update the LinUCB data.
Record the recommendations made in a file "recommendations.tsv" with columns "recipe_slug", "score_bound", and "reward".
The rows in this file should be in the same order as the recommendations were made.

In [4]:
# YOUR CHANGES HERE

Xdf = pd.read_csv("features.tsv", sep="\t")
rdf = pd.read_csv("ratings.tsv", sep="\t").set_index("recipe_slug")

X = Xdf.set_index("recipe_slug")
Xmat = X.to_numpy(dtype=float)
slugs = X.index.tolist()

rating_map = rdf["rating"].to_dict()

alpha_ridge = 1.0
alpha_linucb = 2.0

d = Xmat.shape[1]

A = np.eye(d) * alpha_ridge
b = np.zeros(d)

recs = []

for t in range(len(slugs)):
    A_inv = np.linalg.inv(A)
    theta = A_inv @ b

    scores = []
    for i, slug in enumerate(slugs):
        x = Xmat[i]
        est = x @ theta
        var = np.sqrt(x @ A_inv @ x)
        bound = est + alpha_linucb * var
        scores.append((slug, bound))

    chosen_slug, chosen_bound = max(scores, key=lambda x: x[1])
    i = slugs.index(chosen_slug)

    reward = rating_map[chosen_slug]

    x = Xmat[i]
    A += np.outer(x, x)
    b += reward * x

    recs.append((chosen_slug, chosen_bound, reward))

    slugs.pop(i)
    Xmat = np.delete(Xmat, i, axis=0)

rec_df = pd.DataFrame(recs, columns=["recipe_slug", "score_bound", "reward"])
rec_df.to_csv("recommendations.tsv", sep="\t", index=False)

print(f"Saved recommendations.tsv with shape={rec_df.shape}")
print(rec_df.head())


Saved recommendations.tsv with shape=(100, 3)
        recipe_slug  score_bound  reward
0     apple-crumble     7.483315    0.75
1     ma-la-chicken     7.242589    0.50
2       quesadillas     7.230044    0.50
3             ramen     7.236199    0.00
4  pain-au-chocolat     6.957115    1.00


Submit "recommendations.tsv" in Gradescope.

## Part 7: Acknowledgments

Make a file "acknowledgments.txt" documenting any outside sources or help on this project.
If you discussed this assignment with anyone, please acknowledge them here.
If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for.
If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy.
If no acknowledgements are appropriate, just write none in the file.


Submit "acknowledgments.txt" in Gradescope.

## Part 8: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.


Submit "project.ipynb" in Gradescope.